In [1]:
from preprocessing import *
import os, pickle, csv, copy
import tqdm
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import resample
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from keras_tqdm import TQDMNotebookCallback
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [ ]:
data_Folder = Path('data')  

data_folderIO = data_Folder / "EEG-IO"
data_folderVR = data_Folder / "EEG-VR"
data_folderVV = data_Folder / "EEG-VV"

freq = 250                     
parsedData = 200 * freq  
delta = 125
tam = 250  

## IO ##

In [19]:
list_of_filesIO, list_of_files_labelsIO = listFiles(data_folderIO)

rawDadosIO = read_data(data_Folder, data_folderIO, list_of_filesIO, freq, parsedData)

blinksIO, corruptIO = read_labels(data_folderIO, list_of_files_labelsIO)

rawDadosIO = truncateTime(rawDadosIO)

dataIO = movingAverage(rawDadosIO)

newDataIO = removeCorrupt(dataIO, corruptIO)

datasetBlinkIO, indexesIO = datasetBlink(delta, newDataIO, blinksIO)

noBlinkIO = removeBlink(newDataIO, indexesIO)

datasetNoBlinkIO = datasetNoBlink(noBlinkIO, tam)

In [20]:
datasetBlinkIOUP = resample(datasetBlinkIO, replace=True, n_samples=len(datasetNoBlinkIO)  ,random_state=123)

X_data_IO = np.vstack((datasetBlinkIOUP, datasetNoBlinkIO))
y_data_IO = np.zeros((X_data_IO.shape[0]))
y_data_IO[:datasetBlinkIOUP.shape[0]] = 1

X_data_IO.shape, y_data_IO.shape, datasetBlinkIO.shape, datasetNoBlinkIO.shape, datasetBlinkIOUP.shape

((5788, 250), (5788,), (1114, 250), (2894, 250), (2894, 250))

In [17]:
scaler = MinMaxScaler()
scaler.fit(X_data_IO)
X_data_IO = scaler.transform(X_data_IO)

In [21]:
X_data_IO, y_data_IO = shuffle(X_data_IO, y_data_IO)
X_train_IO, X_test_IO, y_train_IO, y_test_IO = train_test_split(X_data_IO, y_data_IO, test_size=0.33, random_state=42)

IO = [X_train_IO, X_test_IO, y_train_IO, y_test_IO]

#filename = 'trainTestIO'
filename = 'trainTestIO_noscale'
file = open(filename, 'wb')
pickle.dump(IO, file)
file.close()

## VR ##

In [22]:
list_of_filesVR, list_of_files_labelsVR = listFiles(data_folderVR)

rawDadosVR = read_data(data_Folder, data_folderVR, list_of_filesVR, freq, parsedData)

blinksVR, corruptVR = read_labels(data_folderVR, list_of_files_labelsVR)

rawDadosVR = truncateTime(rawDadosVR)

removeSlot1 = list(range(20224,20342))
rawDadosVR[7] = np.delete(rawDadosVR[7], removeSlot1, axis=0)

dataVR = movingAverage(rawDadosVR)

newDataVR = removeCorrupt(dataVR, corruptVR)

datasetBlinkVR, indexesVR = datasetBlink(delta, newDataVR, blinksVR)

noBlinkVR = removeBlink(newDataVR, indexesVR)

datasetNoBlinkVR = datasetNoBlink(noBlinkVR, tam)

In [23]:
datasetBlinkVRUP = resample(datasetBlinkVR, replace=True, n_samples=len(datasetNoBlinkVR)  ,random_state=123)

X_data_VR = np.vstack((datasetBlinkVRUP, datasetNoBlinkVR))
y_data_VR = np.zeros((X_data_VR.shape[0]))
y_data_VR[:datasetBlinkVRUP.shape[0]] = 1

X_data_VR.shape, y_data_VR.shape, datasetBlinkVR.shape, datasetNoBlinkVR.shape, datasetBlinkVRUP.shape

((7132, 250), (7132,), (1342, 250), (3566, 250), (3566, 250))

In [24]:
scaler = MinMaxScaler()
scaler.fit(X_data_VR)
X_data_VR = scaler.transform(X_data_VR)

In [25]:
X_data_VR, y_data_VR = shuffle(X_data_VR, y_data_VR)
X_train_VR, X_test_VR, y_train_VR, y_test_VR = train_test_split(X_data_VR, y_data_VR, test_size=0.33, random_state=42)

VR = [X_train_VR, X_test_VR, y_train_VR, y_test_VR]

filename = 'trainTestVR'
file = open(filename, 'wb')
pickle.dump(VR, file)
file.close()

## VV ##

In [26]:
list_of_filesVV, list_of_files_labelsVV = listFiles(data_folderVV)

rawDadosVV = read_data(data_Folder, data_folderVV, list_of_filesVV, freq, parsedData)

blinksVV, corruptVV = read_labels(data_folderVV, list_of_files_labelsVV)

rawDadosVV = truncateTime(rawDadosVV)

dataVV = movingAverage(rawDadosVV)

newDataVV = removeCorrupt(dataVV, corruptVV)

datasetBlinkVV, indexesVV = datasetBlink(delta, newDataVV, blinksVV)

noBlinkVV = removeBlink(newDataVV, indexesVV)

datasetNoBlinkVV = datasetNoBlink(noBlinkVV, tam)

In [27]:
datasetBlinkVVUP = resample(datasetBlinkVV, replace=True, n_samples=len(datasetNoBlinkVV)  ,random_state=123)

X_data_VV = np.vstack((datasetBlinkVVUP, datasetNoBlinkVV))
y_data_VV = np.zeros((X_data_VV.shape[0]))
y_data_VV[:datasetBlinkVVUP.shape[0]] = 1

X_data_VV.shape, y_data_VV.shape, datasetBlinkVV.shape, datasetNoBlinkVV.shape, datasetBlinkVVUP.shape

((7076, 250), (7076,), (1588, 250), (3538, 250), (3538, 250))

In [28]:
scaler = MinMaxScaler()
scaler.fit(X_data_VV)
X_data_VV = scaler.transform(X_data_VV)

In [29]:
X_data_VV, y_data_VV = shuffle(X_data_VV, y_data_VV)
X_train_VV, X_test_VV, y_train_VV, y_test_VV = train_test_split(X_data_VV, y_data_VV, test_size=0.33, random_state=42)

VV = [X_train_VV, X_test_VV, y_train_VV, y_test_VV]

filename = 'trainTestVV'
file = open(filename, 'wb')
pickle.dump(VV, file)
file.close()

## Load Pickle ##

In [3]:
file = open('trainTestIO_noscale', 'rb')
X_train_IO_noscale, X_test_IO_noscale, y_train_IO_noscale, y_test_IO_noscale = pickle.load(file)
file.close()

file = open('trainTestIO', 'rb')
X_train_IO, X_test_IO, y_train_IO, y_test_IO = pickle.load(file)
file.close()

file = open('trainTestVR', 'rb')
X_train_VR, X_test_VR, y_train_VR, y_test_VR = pickle.load(file)
file.close()

file = open('trainTestVV', 'rb')
X_train_VV, X_test_VV, y_train_VV, y_test_VV = pickle.load(file)
file.close()

In [4]:
np.sum(y_test_IO), y_test_IO.shape[0], np.sum(y_test_VV), y_test_VV.shape[0], np.sum(y_test_VR), y_test_VR.shape[0]

(956.0, 1911, 1140.0, 2336, 1153.0, 2354)

In [5]:
print('IO',np.sum(y_test_IO)/ y_test_IO.shape[0], 1-(np.sum(y_test_IO)/ y_test_IO.shape[0]))
print('VR',np.sum(y_test_VR)/ y_test_VR.shape[0], 1-(np.sum(y_test_VR)/ y_test_VR.shape[0]))
print('VV',np.sum(y_test_VV)/ y_test_VV.shape[0], 1-(np.sum(y_test_VV)/ y_test_VV.shape[0]))

IO 0.500261643118786 0.49973835688121404
VR 0.4898045879354291 0.5101954120645709
VV 0.488013698630137 0.511986301369863


### Ipywidgets display ###

In [6]:
def sort(labels):
    """Sort the indexes of the array passed as parameter in two lists according to the values of the elements corresponding to the index
    
    Parameters
    ----------
    labels : numpy.array
        Array of values 1 or 0
        
    Returns
    -------
    one : list
        Array of indexes corresponding to the elements equals to 1 in the array passed as parameter 
    zero : list 
        Array of indexes corresponding to the elements equals to 0 in the array passed as parameter 
    """
    one = []
    zero = []
    for i in range(len(labels)):
        if labels[i] == 1:
            one.append(i)
        else:
            zero.append(i)
            
    return one, zero

In [7]:
X_testBlink_IO, X_testNo_IO = sort(y_test_IO)
X_testBlink_VR, X_testNo_VR = sort(y_test_VR)
X_testBlink_VV, X_testNo_VV = sort(y_test_VV)

In [8]:
def do_stuff(a, b):
    if a == 'IO':
        plt.plot(X_test_IO[b])
        plt.title(y_test_IO[b])        
    if a == 'VR':
        plt.plot(X_test_VR[b])
        plt.title(y_test_VR[b])
    if a == 'VV':
        plt.plot(X_test_VV[b])
        plt.title(y_test_VV[b])
    return 

def ajuste(*args):
    if data.value == 'IO' and blink.value == 'Blink':
        number.options = X_testBlink_IO
    elif data.value == 'VR' and blink.value == 'Blink':
        number.options = X_testBlink_VR
    elif data.value == 'VV' and blink.value == 'Blink':
        number.options = X_testBlink_VV
    elif data.value == 'IO' and blink.value == 'No Blink':
        number.options = X_testNo_IO
    elif data.value == 'VR' and blink.value == 'No Blink':
        number.options = X_testNo_VR
    elif data.value == 'VV' and blink.value == 'No Blink':
        number.options = X_testNo_VV
    else:
        number.options = []
        
        
data = widgets.Dropdown(
    options=['-', 'IO', 'VR', 'VV'],
    description='Conjunto:',
    disabled=False,
)

blink = widgets.Dropdown(
    options=['Blink', 'No Blink'],
    description='Estado:',
    disabled=False,
)

number = widgets.Dropdown(
    options=[' - '],
    description='Indice:',
    disabled=False,
)        

        
data.observe(ajuste, 'value')
blink.observe(ajuste, 'value')

controls = widgets.HBox([data, blink, number])
graphic = widgets.interactive_output(do_stuff, {'a': data, 'b': number})
display(controls, graphic)

Output()